# Thymio Control System

This notebook contains the main control loop for the Thymio robot, integrating vision, state estimation, and motion control modules implemented in separate files.

### Turn off LEDs

In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()  # MAKE SURE TO CALL notebook.stop() ONCE DONE USING THYMIO

In [2]:
%%run_python
nf_leds_bottom_left(0,0,0)
nf_leds_bottom_right(0,0,0)
nf_leds_prox_v(0, 0)
nf_leds_prox_h(0, 0, 0, 0, 0, 0, 0, 0)
nf_leds_circle(0, 0, 0, 0, 0, 0, 0, 0)
nf_leds_temperature(0, 0)


In [3]:
await tdmclient.notebook.stop()

In [4]:
import numpy as np
import vision
from vision import VisionSystem
#from state_estimation import ExtendedKalmanFilter
from control import ThymioController
from pathfinding import find_path
from utils import Pose, Point, euclidean_distance

In [5]:
import numpy as np
import math
import time
import asyncio

from tdmclient import ClientAsync

from extended_kalman_filter import EKF 
from thymio_math_model import Thymio
from control import ThymioController
from utils import Pose, Point, euclidean_distance

# ===================== Configuración básica ===================== #

INITIAL_POSE = Pose(0.0, 0.0, 0.0)  # x, y, theta en cm y rad
INITIAL_COVARIANCE = np.eye(3) * 1e-1  # Incertidumbre inicial

CAMERA_COVARIANCE = np.diag([
    0.0011857353432198614,
    0.0017873989613467563,
    6.1009773737464586e-05
])  # No la vamos a usar (no hay update de cámara), pero EKF la pide

DT = 0.1 # periodo de control (s)
WAYPOINT_THRESHOLD = 3.0  # cm para considerar waypoint alcanzado

# Waypoints hardcodeados en cm (ajusta a tu entorno)
WAYPOINTS = [
    Point(20.0, 0.0),
    Point(20.0, 20.0),
    Point(0.0, 20.0),
]


# ===================== Helpers ===================== #

def wrap_angle(angle):
    return math.atan2(math.sin(angle), math.cos(angle))


# ===================== Setup modelo + EKF ===================== #

thymio_model = Thymio()
thymio_model.dt = DT
thymio_model.freq = 1.0 / DT

ekf = EKF(
    thymio_model,
    np.array([INITIAL_POSE.x, INITIAL_POSE.y, INITIAL_POSE.theta]),
    INITIAL_COVARIANCE,
    CAMERA_COVARIANCE
)


# ===================== Main loop sin cámara ===================== #

async def run_no_camera():
    controller = ThymioController()

    print("Conectando a Thymio...")
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()

    print("Thymio conectado.")

    # Asegurarnos de tener estas variables disponibles
    await node.wait_for_variables([
        'prox.horizontal',
        'motor.left.target',
        'motor.right.target',
        'motor.left.speed',
        'motor.right.speed'
    ])

    current_waypoint_idx = 0
    l_cmd, r_cmd = 0, 0

    # Inicializa velocidades del modelo
    thymio_model.v_l = 0.0
    thymio_model.v_r = 0.0

    try:
        print("Iniciando loop principal SIN cámara...")

        while True:
            loop_start = time.time()

            # 1) Leer sensores del Thymio (de verdad)
            prox_sensors = list(node.v.prox.horizontal)

            try:
                left_speed_measurement = node.v.motor.left.speed
                right_speed_measurement = node.v.motor.right.speed
            except Exception as e:
                print(f"Error leyendo velocidades de motores: {e}")
                left_speed_measurement = 0
                right_speed_measurement = 0

            # 2) Actualizar modelo Thymio (velocidades en cm/s)
            thymio_model.v_l = thymio_model.thymio_unit_to_speed(left_speed_measurement)
            thymio_model.v_r = thymio_model.thymio_unit_to_speed(right_speed_measurement)

            # 3) EKF: solo predict (SIN update, no hay cámara)
            ekf.predict_step()

            est_vec = ekf.x.flatten()
            estimated_pose = Pose(
                float(est_vec[0]),
                float(est_vec[1]),
                float(est_vec[2])
            )

            print(
                f"EKF Pose: x={estimated_pose.x:.2f} cm, "
                f"y={estimated_pose.y:.2f} cm, "
                f"theta={estimated_pose.theta:.2f} rad, "
                f"v_l={thymio_model.v_l:.2f} cm/s, v_r={thymio_model.v_r:.2f} cm/s"
            )

            # 4) Lógica de waypoints hardcodeados
            if current_waypoint_idx < len(WAYPOINTS):
                target = WAYPOINTS[current_waypoint_idx]

                dist_to_wp = euclidean_distance(
                    Point(estimated_pose.x, estimated_pose.y),
                    target
                )

                if dist_to_wp < WAYPOINT_THRESHOLD:
                    print(f"Reached Waypoint {current_waypoint_idx} @ ({target.x:.1f}, {target.y:.1f})")
                    current_waypoint_idx += 1

                if current_waypoint_idx >= len(WAYPOINTS):
                    print("GOAL REACHED! Stopping.")
                    break

                # 5) CONTROL (usa tu ThymioController de siempre)
                l_cmd, r_cmd = controller.update(
                    estimated_pose,
                    target,
                    prox_sensors
                )
            else:
                # Por si acaso
                l_cmd, r_cmd = 0, 0
                print("No more waypoints, should stop.")
                break

            # 6) Enviar comandos al Thymio (DE VERDAD)
            v = {
                "motor.left.target": [int(l_cmd)],
                "motor.right.target": [int(r_cmd)],
            }
            print(f"(l_cmd, r_cmd): ({int(l_cmd)}, {int(r_cmd)})")
            await node.set_variables(v)
            node.flush()

            # 7) Mantener frecuencia ~DT
            elapsed = time.time() - loop_start
            if elapsed < DT:
                await client.sleep(DT - elapsed)

    finally:
        print("Stopping motors and unlocking Thymio...")
        v = {
            "motor.left.target": [0],
            "motor.right.target": [0],
        }
        await node.set_variables(v)
        node.flush()
        await node.unlock()
        print("Thymio disconnected.")





In [6]:
try:
    await run_no_camera() 
except KeyboardInterrupt:
    print("Program interrupted by user. Exiting...")

Conectando a Thymio...
Thymio conectado.
Iniciando loop principal SIN cámara...
[EKF] ENTER predict_step, x = [0. 0. 0.]
[EKF] EXIT predict_step, x = [0. 0. 0.]
EKF Pose: x=0.00 cm, y=0.00 cm, theta=0.00 rad, v_l=0.00 cm/s, v_r=0.00 cm/s
(l_cmd, r_cmd): (120, 120)
[EKF] ENTER predict_step, x = [0. 0. 0.]
[EKF] EXIT predict_step, x = [0. 0. 0.]
EKF Pose: x=0.00 cm, y=0.00 cm, theta=0.00 rad, v_l=0.00 cm/s, v_r=0.00 cm/s
(l_cmd, r_cmd): (120, 120)
[EKF] ENTER predict_step, x = [0. 0. 0.]
[EKF] EXIT predict_step, x = [0.3223125  0.         0.00177632]
EKF Pose: x=0.32 cm, y=0.00 cm, theta=0.00 rad, v_l=3.14 cm/s, v_r=3.31 cm/s
(l_cmd, r_cmd): (120, 119)
[EKF] ENTER predict_step, x = [0.3223125  0.         0.00177632]
[EKF] EXIT predict_step, x = [6.80061936e-01 6.35476639e-04 2.48684211e-03]
EKF Pose: x=0.68 cm, y=0.00 cm, theta=0.00 rad, v_l=3.54 cm/s, v_r=3.61 cm/s
(l_cmd, r_cmd): (120, 119)
[EKF] ENTER predict_step, x = [6.80061936e-01 6.35476639e-04 2.48684211e-03]
[EKF] EXIT predict_

In [7]:
import numpy as np
import math
import time
import asyncio

from tdmclient import ClientAsync

from thymio_math_model import Thymio
from control import ThymioController
from utils import Pose, Point, euclidean_distance

# ===================== Configuración básica ===================== #

INITIAL_POSE = Pose(0.0, 0.0, 0.0)  # x, y, theta en cm y rad

DT = 0.1  # periodo de control (s)
WAYPOINT_THRESHOLD = 3.0  # cm para considerar waypoint alcanzado

# Waypoints hardcodeados en cm (ajusta a tu entorno)
WAYPOINTS = [
    Point(20.0, 0.0),
    Point(20.0, 20.0),
    Point(0.0, 20.0),
]


# ===================== Helpers ===================== #

def wrap_angle(angle):
    return math.atan2(math.sin(angle), math.cos(angle))


# ===================== Setup modelo (solo odometría) ===================== #

thymio_model = Thymio()
thymio_model.dt = DT
thymio_model.freq = 1.0 / DT

thymio_model.sigma_x_model, thymio_model.sigma_y_model = 0.05


# ===================== Main loop sin EKF ni cámara ===================== #

async def run_no_camera_pure_odom():
    controller = ThymioController()

    print("Conectando a Thymio...")
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()

    print("Thymio conectado.")

    # Asegurarnos de tener estas variables disponibles
    await node.wait_for_variables([
        'prox.horizontal',
        'motor.left.target',
        'motor.right.target',
        'motor.left.speed',
        'motor.right.speed'
    ])

    current_waypoint_idx = 0
    l_cmd, r_cmd = 0, 0

    # Inicializa velocidades del modelo
    thymio_model.v_l = 0.0
    thymio_model.v_r = 0.0

    # Pose estimada por pura odometría
    estimated_pose = Pose(INITIAL_POSE.x, INITIAL_POSE.y, INITIAL_POSE.theta)

    try:
        print("Iniciando loop principal SIN EKF (solo odometría)...")

        while True:
            loop_start = time.time()

            # 1) Leer sensores del Thymio (de verdad)
            prox_sensors = list(node.v.prox.horizontal)

            try:
                left_speed_measurement = node.v.motor.left.speed
                right_speed_measurement = node.v.motor.right.speed
            except Exception as e:
                print(f"Error leyendo velocidades de motores: {e}")
                left_speed_measurement = 0
                right_speed_measurement = 0

            # 2) Actualizar modelo Thymio (velocidades en cm/s)
            thymio_model.v_l = thymio_model.thymio_unit_to_speed(left_speed_measurement)
            thymio_model.v_r = thymio_model.thymio_unit_to_speed(right_speed_measurement)

            # 3) ODOMETRÍA PURA: integrar la pose con el modelo diferencial
            v_l = thymio_model.v_l
            v_r = thymio_model.v_r
            L = thymio_model.L  # distancia entre ruedas (cm)

            v = 0.5 * (v_r + v_l)            # velocidad lineal (cm/s)
            omega = (v_r - v_l) / L          # velocidad angular (rad/s)

            # Integración simple (Euler hacia adelante)
            new_x = estimated_pose.x + v * math.cos(estimated_pose.theta) * DT
            new_y = estimated_pose.y + v * math.sin(estimated_pose.theta) * DT
            new_theta = wrap_angle(estimated_pose.theta + omega * DT)

            # >>> AQUÍ EL CAMBIO IMPORTANTE: recrear la Pose, no modificar atributos <<<
            estimated_pose = Pose(new_x, new_y, new_theta)

            print(
                f"ODOM Pose: x={estimated_pose.x:.2f} cm, "
                f"y={estimated_pose.y:.2f} cm, "
                f"theta={estimated_pose.theta:.2f} rad, "
                f"v_l={v_l:.2f} cm/s, v_r={v_r:.2f} cm/s"
            )

            # 4) Lógica de waypoints hardcodeados
            if current_waypoint_idx < len(WAYPOINTS):
                target = WAYPOINTS[current_waypoint_idx]

                dist_to_wp = euclidean_distance(
                    Point(estimated_pose.x, estimated_pose.y),
                    target
                )

                if dist_to_wp < WAYPOINT_THRESHOLD:
                    print(f"Reached Waypoint {current_waypoint_idx} @ ({target.x:.1f}, {target.y:.1f})")
                    current_waypoint_idx += 1

                if current_waypoint_idx >= len(WAYPOINTS):
                    print("GOAL REACHED! Stopping.")
                    break

                # 5) CONTROL (usa tu ThymioController de siempre)
                l_cmd, r_cmd = controller.update(
                    estimated_pose,
                    target,
                    prox_sensors
                )
            else:
                # Por si acaso
                l_cmd, r_cmd = 0, 0
                print("No more waypoints, should stop.")
                break

            # 6) Enviar comandos al Thymio (DE VERDAD)
            v_out = {
                "motor.left.target": [int(l_cmd)],
                "motor.right.target": [int(r_cmd)],
            }
            print(f"(l_cmd, r_cmd): ({int(l_cmd)}, {int(r_cmd)})")
            await node.set_variables(v_out)
            node.flush()

            # 7) Mantener frecuencia ~DT
            elapsed = time.time() - loop_start
            if elapsed < DT:
                await client.sleep(DT - elapsed)

    finally:
        print("Stopping motors and unlocking Thymio...")
        v_stop = {
            "motor.left.target": [0],
            "motor.right.target": [0],
        }
        await node.set_variables(v_stop)
        node.flush()
        await node.unlock()
        print("Thymio disconnected.")


# Si quieres ejecutarlo directo desde este script:
# if __name__ == "__main__":
#     asyncio.run(run_no_camera_pure_odom())


TypeError: cannot unpack non-iterable float object

In [8]:
try:
    await run_no_camera_pure_odom() 
except KeyboardInterrupt:
    print("Program interrupted by user. Exiting...")

Conectando a Thymio...
Thymio conectado.
Iniciando loop principal SIN EKF (solo odometría)...
ODOM Pose: x=0.00 cm, y=0.00 cm, theta=0.00 rad, v_l=0.00 cm/s, v_r=0.00 cm/s
(l_cmd, r_cmd): (120, 120)
ODOM Pose: x=0.00 cm, y=0.00 cm, theta=0.00 rad, v_l=0.00 cm/s, v_r=0.00 cm/s
(l_cmd, r_cmd): (120, 120)
ODOM Pose: x=0.03 cm, y=0.00 cm, theta=0.01 rad, v_l=0.00 cm/s, v_r=0.68 cm/s
(l_cmd, r_cmd): (120, 119)
ODOM Pose: x=0.07 cm, y=0.00 cm, theta=0.01 rad, v_l=0.00 cm/s, v_r=0.68 cm/s
(l_cmd, r_cmd): (121, 118)
ODOM Pose: x=0.10 cm, y=0.00 cm, theta=0.02 rad, v_l=0.00 cm/s, v_r=0.68 cm/s
(l_cmd, r_cmd): (122, 117)
ODOM Pose: x=0.47 cm, y=0.01 cm, theta=0.02 rad, v_l=3.54 cm/s, v_r=3.75 cm/s
(l_cmd, r_cmd): (122, 117)
ODOM Pose: x=0.83 cm, y=0.02 cm, theta=0.03 rad, v_l=3.54 cm/s, v_r=3.75 cm/s
(l_cmd, r_cmd): (122, 117)
ODOM Pose: x=1.19 cm, y=0.03 cm, theta=0.03 rad, v_l=3.54 cm/s, v_r=3.75 cm/s
(l_cmd, r_cmd): (122, 117)
ODOM Pose: x=1.62 cm, y=0.04 cm, theta=0.03 rad, v_l=4.29 cm/s, v_

In [ ]:
##### just to get the visu
import vision
from vision import VisionSystem
from pathfinding import find_path
from utils import Point
import importlib
importlib.reload(vision)
import time

try: del vis
except NameError: pass
vis = VisionSystem(camera_index=1, warmup_time=3)
graph, start_node, goal_node = vis.construct_map(cspace_padding=2.0)
waypoints_idx = find_path(graph, start_node, goal_node)
waypoints: list[Point] = [graph.nodes[i]['pos'] for i in waypoints_idx]
vis.live_visu(graph, start_node, goal_node, waypoints, resolution=(1600, 900))



In [ ]:
while True:
    if vis.update_robot_visu():  # returns True if 'q' pressed
        break
    time.sleep(0.05)

### Defining Main Runner Function

In [5]:
import numpy as np
from tdmclient import ClientAsync
import sys
import time

from vision import VisionSystem 
from control import ThymioController
from pathfinding import find_path
from utils import Pose, Point, euclidean_distance

from extended_kalman_filter import EKF 
from thymio_math_model import Thymio 



async def run(vis: VisionSystem):

    INITIAL_POSE = Pose(0, 0, 0)  # x, y, theta
    INITIAL_COVARIANCE = np.eye(3) * 1e-1  # Initial uncertainty in state estimation
    CAMERA_COVARIANCE = np.diag([
            0.0011857353432198614,
            0.0017873989613467563,
            6.1009773737464586e-05
        ]) # These are from the webcam, they may work but ideally change to Tanc's phone


    WAYPOINT_THRESHOLD = 3.0  # cm
    DT = 0.1  # Control loop time step in seconds (nominal)

    thymio = Thymio()
    thymio.dt = DT
    thymio.freq = 1.0 / DT

    


    ekf = EKF(thymio, 
                np.array([INITIAL_POSE.x, INITIAL_POSE.y, INITIAL_POSE.theta]), 
                INITIAL_COVARIANCE,
                CAMERA_COVARIANCE 
            )

    #ekf = ExtendedKalmanFilter(initial_pose=INITIAL_POSE, initial_covariance=INITIAL_COVARIANCE)
    controller = ThymioController()
    
    # B. Map & Plan
    graph, start_node, goal_node = vis.construct_map(cspace_padding=2.0)
    waypoints_idx = find_path(graph, start_node, goal_node)
    waypoints: list[Point] = [graph.nodes[i]['pos'] for i in waypoints_idx]
    
    vis.init_visu(graph, start_node, goal_node, waypoints, resolution=(1400,1000))
    
    current_waypoint_idx = 0
    l_cmd, r_cmd = 0, 0  # Initialize command speeds

    thymio.v_l, thymio.v_r = 0, 0   # Set initial speed commands for thymio_model

    #ekf.predict_step()
    #ekf.update_step()
    
    print("Connectin to Thymio...")

    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()
    #await node.stop()

    print("Thymio connected.")

    await node.wait_for_variables([
                'prox.horizontal',
                'motor.left.target',
                'motor.right.target',
                'motor.left.speed',
                'motor.right.speed'
                ])
    
    # Loop

    #await node.set_variables({"motor.left.target":[0], "motor.right.target":[0]})
    
    # ---------- REAL-TIME DT INIT ----------
    last_time = time.time()
    # --------------------------------------
    
    try:
        print("Starting main loop...")

        while True:
            start_time = time.time()            

            # ---------- REAL-TIME DT UPDATE ----------
            now = time.time()
            dt_real = now - last_time
            last_time = now

            # por si acaso, acotar dt para que no explote nada
            if dt_real <= 0:
                dt_real = DT
            elif dt_real > 0.5:
                dt_real = 0.5

            thymio.dt = dt_real
            thymio.freq = 1.0 / dt_real

            # debug opcional:
            print(f"dt_real: {dt_real:.3f} s")
            # -----------------------------------------

            prox_sensors = list(node.v.prox.horizontal)
            # print("Prox Sensors:", prox_sensors)

            # 1. Pose Measurement & Sensing
            vision_pose_measurement = vis.get_robot_pose() # May be None
            #print("Vision Pose Measurement:", vision_pose_measurement)
            
            try:
                left_speed_measurement = node.v.motor.left.speed #target
                right_speed_measurement = node.v.motor.right.speed #target
                print("Speed Updated")

            except Exception as e:
                print(f"Error reading motor speeds: {e}")
                left_speed_measurement = 0
                right_speed_measurement = 0

            
            thymio.v_l = thymio.thymio_unit_to_speed(left_speed_measurement)
            thymio.v_r = thymio.thymio_unit_to_speed(right_speed_measurement)

            print(thymio.v_l, thymio.v_r)
            
            # 2. State Estimation via EKF
            if ekf.predict_step():
                print("Predict done")
                print(f"Current EKF Pose: {ekf.x[0], ekf.x[1], ekf.x[2]}")
            else: 
                print("Not even touched")

            #print(f"EKF: predict: {ekf.x} while speed readings are: {thymio.vl}, {thymio.v_r}")

            #ekf.predict(robot.left_speed, robot.right_speed, dt)
            #if vision_pose_measurement:
                #ekf.update(vision_pose_measurement)

            if vision_pose_measurement is not None: 
                z = np.array([
                    vision_pose_measurement.x,
                    vision_pose_measurement.y, 
                    vision_pose_measurement.theta
                ], dtype=float)

                #print(f"Vision gets: {vision_pose_measurement}, speed: {thymio.v_l}, {thymio.v_r}")
                
                if ekf.update_step(z): 
                    print("Update done")
                else: 
                    print("Updaten't")
            else: 
                print("No pose estimate available, skipping update.")
                #continue

            # EKF estimated pose to Pose()
                
            est_vec = ekf.x.flatten() # vision_pose_measurement#ekf.get_state()
            estimated_pose = Pose(
                 est_vec[0],
                 est_vec[1],
                 est_vec[2]
                 )
            
            
            print(f"EKF gets: {est_vec[0], est_vec[1], est_vec[2]}")
            #print(f"EKF Var: {ekf.P}")
                        
            
            # 3. PATH LOGIC
            if current_waypoint_idx < len(waypoints):
                target = waypoints[current_waypoint_idx]
                
                # Check if reached waypoint
                dist_to_wp = euclidean_distance(Point(estimated_pose.x, estimated_pose.y), target) # *estimated_pose[:2]
                if dist_to_wp < WAYPOINT_THRESHOLD:
                    print(f"Reached Waypoint {current_waypoint_idx}!")
                    current_waypoint_idx += 1
                    
                # Check if reached GOAL
                if current_waypoint_idx >= len(waypoints):
                    print("GOAL REACHED! Stopping.")
                    break # Exit loop
                
                # 4. CONTROL
                # Get commands from your controller
                # Ensure controller returns integers! Thymio expects ints.
                l_cmd, r_cmd = controller.update(
                    estimated_pose,
                    target,
                    prox_sensors
                    )

            # 5. SEND COMMANDS (Async)
            # We write to the variables. This doesn't send yet.
            #node.v.motor.left.target = int(l_cmd)
            #node.v.motor.right.target = int(r_cmd)
            v = {
                    "motor.left.target": [int(l_cmd)],
                    "motor.right.target": [int(r_cmd)],
                }
            print(f"(l_cmd, r_cmd): ({int(l_cmd)}, {int(r_cmd)})")
            await node.set_variables(v)
            
            # Force send the command packet immediately
            node.flush()
            
            # 6. LOOP TIMING 
            # Calculate how much time the logic took, and sleep the remainder
            # to maintain a steady frequency (e.g., 10Hz)
            elapsed = time.time() - start_time
            if elapsed < DT:
                await client.sleep(DT - elapsed)
            
            # Update visu
            if vis.update_robot_visu():  # returns True if 'q' pressed
                print("User requested exit via visu.")
                break

    finally:
        print("Stopping motors and unlocking Thymio...")
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        await node.set_variables(v)
        node.flush()
        await node.unlock()

        print("Thymio disconnected.")
        while True:
            if vis.update_robot_visu():  # returns True if 'q' pressed
                break
            await client.sleep(0.05)


### Running

##### Vision

In [22]:
try: del vis
except NameError: pass

Camera released.


In [23]:
# Initialize vision system before running (and only once)
vis = VisionSystem(camera_index=1, warmup_time=10)

Warming up camera for 10s or until exposure settles...
Camera ready. (Resolution: 1920.0x1080.0)


In [ ]:
# Actually run the control system:
try:
    await run(vis)
except KeyboardInterrupt:
    print("Program interrupted by user. Exiting...")

Point(x=np.float32(13.119865), y=np.float32(15.901657))
Connectin to Thymio...
Thymio connected.
Starting main loop...
dt_real: 0.100 s
Speed Updated
0.0 0.0
[EKF] ENTER predict_step, x = [0. 0. 0.]
[EKF] EXIT predict_step, x = [0. 0. 0.]
Predict done
Current EKF Pose: (array([0.]), array([0.]), array([0.]))
[EKF] ENTER update_step, z = [13.11986542 15.9016571   1.93136233]
[EKF] EXIT update_step, x = [12.96748846 15.62486944  1.9302917 ]
Update done
EKF gets: (np.float64(12.967488461435186), np.float64(15.624869439406227), np.float64(1.9302917049569974))
Reached Waypoint 0!
(l_cmd, r_cmd): (164, -8)
dt_real: 0.176 s
Speed Updated
1.0462500000000001 0.0
[EKF] ENTER predict_step, x = [12.96748846 15.62486944  1.9302917 ]
[EKF] EXIT predict_step, x = [12.935132   15.71096351  1.91092886]
Predict done
Current EKF Pose: (array([12.935132]), array([15.71096351]), array([1.91092886]))
[EKF] ENTER update_step, z = [13.02598476 16.53591156  1.80453951]
[EKF] EXIT update_step, x = [12.98983201 

Exception in thread Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-4:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Thread-8:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\Usuario\AppData\Roaming\Python\Python312\site-packages\tdmclient\tcp.py", line 73, in run
    self.run()
  File "C:\Users\Usuario\AppData\Roaming\Python\Python312\site-packages\tdmclient\tcp.py", line 73, in run
    se

### Separate Vison Tests

In [ ]:
import matplotlib.pyplot as plt

from vision import VisionSystem
from pathfinding import find_path


try: del vis
except NameError: pass
vis = VisionSystem(camera_index=1)

print(vis.get_robot_pose())
plt.imshow(vis.warp_image(vis.img))


# Build graph
g, start, end = vis.construct_map()

# Run the pathfinding
waypoints = find_path(g, start, end)
waypoints = [g.nodes[n]['pos'] for n in waypoints]  # Convert node IDs to world-space

# Visualize the resulting visibility graph
plt.figure(figsize=(16, 10))

# Add the warped image as background to enforce image coordinates (0,0 at top-left)
plt.imshow(vis.warp_image(vis.img)[::-1])

# Since coordinates have been converted to world-space with (0,0) at bottom-left,
# we need to unflip the y-axis (caused by imshow)
plt.gca().invert_yaxis()

# Plot all edges
for u, v in g.edges():
    p1 = g.nodes[u]['pos']
    p2 = g.nodes[v]['pos']
    xs = [p1[0] * vis.pxl_per_cm_x, p2[0] * vis.pxl_per_cm_x]
    ys = [p1[1] * vis.pxl_per_cm_y, p2[1] * vis.pxl_per_cm_y]
    plt.plot(xs, ys, color="lightsalmon", alpha=0.4, linewidth=1)

# Plot the waypoints path
for i in range(len(waypoints) - 1):
    p1 = waypoints[i]
    p2 = waypoints[i + 1]
    xs = [p1[0] * vis.pxl_per_cm_x, p2[0] * vis.pxl_per_cm_x]
    ys = [p1[1] * vis.pxl_per_cm_y, p2[1] * vis.pxl_per_cm_y]
    plt.plot(xs, ys, color="deepskyblue", alpha=0.6, linewidth=5, label="Planned Path" if i == 0 else "")

# Plot all nodes
for i in g.nodes():
    pt = g.nodes[i]['pos']
    color = "deepskyblue" if pt in waypoints else "coral"
    size = 70 if pt in waypoints else 30
    plt.scatter(pt[0] * vis.pxl_per_cm_x, pt[1] * vis.pxl_per_cm_y, color=color, s=size, alpha=0.8, marker="o")

# Highlight start and goal nodes
start_pos = g.nodes[start]['pos']
goal_pos = g.nodes[end]['pos']
plt.scatter(start_pos[0] * vis.pxl_per_cm_x, start_pos[1] * vis.pxl_per_cm_y, color="limegreen", s=120, marker="o", label="Start")
plt.scatter(goal_pos[0] * vis.pxl_per_cm_x, goal_pos[1] * vis.pxl_per_cm_y, color="deeppink", s=120, marker="o", label="Goal")

plt.title("Visibility Graph with Planned Path from Package")
plt.legend()
plt.axis("off")
plt.show()

print(vis.get_robot_pose())
print(vis.get_robot_pose())

In [ ]:
for i in range(20):
    print(f"{vis.get_robot_pose() = }")